In [17]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time
import json

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [18]:
def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-04-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-04-30", "%Y-%m-%d")

#Create a range of dates to iterate 
#Note: Periods here represents the number of days it will create from the start date 
#We also do a +2 since it will only generate up to April 29. We inlcude May 1 
#since we want to get data from the last day which is April 30 to May 1 
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())

#prepare the parameters needed to call the API
sort_type="score"
sort="desc"
fields=["id","author","subreddit","created_utc","num_comments","score","title","selftext"]
subreddit = 'aww'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)

Doing 2020-04-01 to 2020-04-02
=====Done
Doing 2020-04-02 to 2020-04-03
=====Done
Doing 2020-04-03 to 2020-04-04
=====Done
Doing 2020-04-04 to 2020-04-05
=====Done
Doing 2020-04-05 to 2020-04-06
=====Done
Doing 2020-04-06 to 2020-04-07
=====Done
Doing 2020-04-07 to 2020-04-08
=====Done
Doing 2020-04-08 to 2020-04-09
=====Done
Doing 2020-04-09 to 2020-04-10
=====Done
Doing 2020-04-10 to 2020-04-11
=====Done
Doing 2020-04-11 to 2020-04-12
=====Done
Doing 2020-04-12 to 2020-04-13
=====Done
Doing 2020-04-13 to 2020-04-14
=====Done
Doing 2020-04-14 to 2020-04-15
=====Done
Doing 2020-04-15 to 2020-04-16
=====Done
Doing 2020-04-16 to 2020-04-17
=====Done
Doing 2020-04-17 to 2020-04-18
=====Done
Doing 2020-04-18 to 2020-04-19
=====Done
Doing 2020-04-19 to 2020-04-20
=====Done
Doing 2020-04-20 to 2020-04-21
=====Done
Doing 2020-04-21 to 2020-04-22
=====Done
Doing 2020-04-22 to 2020-04-23
=====Done
Doing 2020-04-23 to 2020-04-24
=====Done
Doing 2020-04-24 to 2020-04-25
=====Done
Doing 2020-04-25

In [19]:
results

[[{'author': 'canadianhifive',
   'created_utc': 1585701597,
   'id': 'fsp400',
   'num_comments': 205,
   'score': 67,
   'selftext': '',
   'subreddit': 'aww',
   'title': 'Picked up our new isolation buddy!'},
  {'author': 'NikKnack1313',
   'created_utc': 1585702246,
   'id': 'fspaex',
   'num_comments': 204,
   'score': 57,
   'selftext': '',
   'subreddit': 'aww',
   'title': 'My foster pupper is overjoyed to announce his promotion to small branch manager.'},
  {'author': 'FormalExperience',
   'created_utc': 1585707987,
   'id': 'fsqsvl',
   'num_comments': 22,
   'score': 56,
   'selftext': '',
   'subreddit': 'aww',
   'title': 'And...Squeeeeezzzzee'},
  {'author': 'MissesIncomplete',
   'created_utc': 1585704053,
   'id': 'fsprs0',
   'num_comments': 166,
   'score': 52,
   'selftext': '',
   'subreddit': 'aww',
   'title': "I crochet sweaters for dogs and it makes me really happy. I hope seeing my friend's dog Lily in her sweater makes you happy on this lonely Tuesday."},
  

In [20]:
flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty lists is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head())

,author,created_utc,id,num_comments,score,selftext,subreddit,title
0,canadianhifive,1585701597,fsp400,205,67,,aww,Picked up our new isolation buddy!
1,NikKnack1313,1585702246,fspaex,204,57,,aww,My foster pupper is overjoyed to announce his ...
2,FormalExperience,1585707987,fsqsvl,22,56,,aww,And...Squeeeeezzzzee
3,MissesIncomplete,1585704053,fsprs0,166,52,,aww,I crochet sweaters for dogs and it makes me re...
4,sacrecoeur1206,1585706416,fsqe7q,34,49,,aww,A goat munching on hydrangeas


In [21]:
# How many submissions were you able to gather?
# Answer: 3000

df["id"].count()

3000

In [22]:
# Who has the most submissions?
# Answer: [deleted]

df["author"].value_counts()

[deleted]             33
TeisTom               22
d3333p7               20
mac_is_crack          17
Thund3rbolt           14
                      ..
Firefox12p             1
prayingatheistintx     1
Kara-El                1
catusjuice             1
Googleitt_             1
Name: author, Length: 2529, dtype: int64

In [23]:
# Which submission has the highest score? 
# Answer: Made her a bridge to the window she always sta...

df_max_score = df[df.score==df["score"].max()]
# df_max_score
df_max_score["title"]

1400    Made her a bridge to the window she always sta...
Name: title, dtype: object

In [24]:
# Which submission has the highest number of comments? 
# Answer: Left my cat alone with a camera for 30 minutes...

# df["num_comments"].max()

df_max_comments = df[df.num_comments==df["num_comments"].max()]
# df_max_comments
df_max_comments["title"]

1301    Left my cat alone with a camera for 30 minutes...
Name: title, dtype: object

In [25]:
# Which day of the week has the most submissions?
# Answer: Wednesdsay (2) and Thursday (3)

df = pd.to_datetime(df["created_utc"], unit='s')
df

0      2020-04-01 00:39:57
1      2020-04-01 00:50:46
2      2020-04-01 02:26:27
3      2020-04-01 01:20:53
4      2020-04-01 02:00:16
               ...        
2995   2020-04-30 23:39:02
2996   2020-04-30 23:47:14
2997   2020-04-30 21:49:31
2998   2020-04-30 22:26:35
2999   2020-04-30 21:45:30
Name: created_utc, Length: 3000, dtype: datetime64[ns]

In [26]:
df.dt.dayofweek

0       2
1       2
2       2
3       2
4       2
       ..
2995    3
2996    3
2997    3
2998    3
2999    3
Name: created_utc, Length: 3000, dtype: int64

In [27]:
df.dt.dayofweek.value_counts()

3    500
2    500
5    400
1    400
6    400
4    400
0    400
Name: created_utc, dtype: int64